In [1]:
import pickle
import torch
import numpy as np
import IPython.display as ipd

In [2]:
from math import ceil
from librosa.filters import mel
from librosa.util import normalize
from melgan.interface import *

In [3]:
from factory.AutoVC import AutoVC
from factory.MetaConv import MetaConv
from factory.AutoVC_Adjust import AutoVC_Adjust

In [4]:
LEN_CROP = 176
DIM_NECK = 44
DIM_EMB = 256
D_MODEL = 80
FREQ = 22
BATCH_SIZE = 2
DEVICE = 'cpu'
ROOT_DIR = 'spmel'

In [5]:
#org = AutoVC(44,256,512,22).to(DEVICE)
#org.load_state_dict(torch.load("model/autovc_pytorch_org_tw.pt", map_location=DEVICE))
#org_dc = AutoVC(44,256,512,22).to(DEVICE)
#org_dc.load_state_dict(torch.load("model/autovc_pytorch_org_dc_tw.pt", map_location=DEVICE))
#conv = MetaConv(44,512,22).to(DEVICE) 
#conv.load_state_dict(torch.load("model/autovc_pytorch_meta_conv_final_tw.pt", map_location=DEVICE))

org_ad = AutoVC_Adjust(44,256,512,22).to(DEVICE)
org_ad.load_state_dict(torch.load("model/autovc_adjust_en.pt", map_location=DEVICE))

<All keys matched successfully>

In [6]:
vocoder = MelVocoder(model_name='model/static/multi_speaker')

In [7]:
metadata = pickle.load(open(f'{ROOT_DIR}/train.pkl', "rb"))

In [8]:
# 馮翊綱,視網膜 -> '柯文哲
#source = 23
#target = 12
#sound = 9

In [9]:
source = 18
target = 12
sound = 4

In [10]:
metadata[source][0],metadata[target][0]

('p245', 'p238')

In [11]:
emb_org = np.expand_dims(metadata[source][1],axis=0)
emb_org.shape

(1, 256)

In [12]:
uttr_org = np.load(f"{ROOT_DIR}//{metadata[source][sound]}")[50:226]
uttr_org  = np.expand_dims(uttr_org ,axis=0)
uttr_org.shape

(1, 176, 80)

In [13]:
emb_trg =  np.expand_dims(metadata[target][1],axis=0)
emb_trg.shape

(1, 256)

## 原始聲音

In [14]:
audio_s = vocoder.inverse(torch.from_numpy(np.transpose(uttr_org,(0,2,1))))
ipd.Audio(audio_s,rate = 22050)

## 目標聲音

In [15]:
uttr_target = np.expand_dims(np.load(f"{ROOT_DIR}//{metadata[target][sound]}").T,axis=0)
audio_t = vocoder.inverse(torch.from_numpy(uttr_target))
ipd.Audio(audio_t,rate = 22050)

In [16]:
#_,res_org,_ = vocoder.inverse(org(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))[1][0,:,:].transpose(1,2))
#_,res_orgdc,_ = vocoder.inverse(org_dc(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))[1][0,:,:].transpose(1,2))
#_,res_conv2,_ = vocoder.inverse(conv2(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))[1][0,:,:].transpose(1,2))
#_,res_conv,_ = vocoder.inverse(conv(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))[1][0,:,:].transpose(1,2))
orgad_audio = vocoder.inverse(org_ad(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))[2][0,:,:].transpose(1,2))

## 轉換聲音

In [17]:
ipd.Audio(orgad_audio,rate = 22050)

In [18]:
#ipd.Audio(conv_audio,rate = 22050)

In [19]:
#ipd.Audio(org_audio,rate = 22050)

In [20]:
#ipd.Audio(orgdc_audio,rate = 22050)

In [21]:
#from scipy.io.wavfile import write
#def save_to_result(audio,filename, rate=22050):
#    write(filename, rate,audio)

In [22]:
#save_to_result(audio_s.squeeze().numpy(),f'speaker_{metadata[source][0]}.wav')

In [23]:
#save_to_result(audio_t.squeeze().numpy(),f'speaker_{metadata[target][0]}.wav')

In [24]:
#save_to_result(org_audio.squeeze().numpy(),f'org_{metadata[source][0]}_{metadata[target][0]}.wav')
#save_to_result(orgdc_audio.squeeze().numpy(),f'orgdc_{metadata[source][0]}_{metadata[target][0]}.wav')
#save_to_result(conv3_audio.squeeze().numpy(),f'conv3_{metadata[source][0]}_{metadata[target][0]}.wav')
#save_to_result(pool_audio.squeeze().numpy(),f'pool_{metadata[source][0]}_{metadata[target][0]}.wav')

In [25]:
#metadata[target][0]

In [26]:
#save_to_result(conv3_audio.squeeze().numpy(),f'conv3_{metadata[source][0]}_{metadata[target][0]}.wav')